In [ ]:
import requests
import cv2
import json
import numpy as np
import pandas as pd


In [65]:
# Импортируем api ключи
from api_keys import api_Geosajest, api_Geocoder, api_static
api_Geosajest = 'e9f68af6-5915-4146-a4fc-aaf7a9a295fa'

In [ ]:
df = pd.read_csv('select-database.csv')

coordinate_arr = df['coordinate']

In [ ]:
spn = '0.02,0.02'
distance = 1500
CenterCoordinate = '37.617698, 55.755864'

infoMap = {
    'metro': ['метро', 'metro'],
    'cafe': ['Где%20поесть', 'biz'],
    'park': ['парк', 'biz'],
    'beaches': ['пляж', 'biz'],
    'hospital': ['больница, поликлиника', 'biz'],
    'police': ['полиция', 'biz'],
    'shop': ['Продукты', 'biz'],
    'beautySalone': ['Салоны%красоты', 'biz'],
    'mall': ['Торговые%центры', 'biz'],
    'museum': ['Музеи', 'biz'],
    'pharmacy': ['Аптеки', 'biz'],
    'sportCenter': ['Спорт', 'biz'],
    'postOffice': ['почта', 'biz'],
    'hotel': ['Гостиницы', 'biz'],
    'college': ['колледж', 'biz'],
    'school': ['школы', 'biz'],
    'kindergarten': ['Детский сад, ясли', 'biz']
}


# sportCenter,postOffice,hotel,college,school,kindergarten

#метро - metro
#парк - biz
#рестораны/каффе - Где%20поесть/Кафе - biz
#Гостиницы - biz

Запрос к карте для получения её изображения

In [ ]:
# Создаём функцию запроса для получения карты Яндекс
def requests_map(ll,spn,api):
    response = requests.get(f'https://static-maps.yandex.ru/v1?ll={ll}&spn={spn}&apikey={api}') #ll = 28.978178,41.011218 / spn = 0.01,0.01 / api = api_static
    if response.status_code == 200:
        print("success")
        return response
    else:
        print("error:", str(response.status_code))
    

In [ ]:
# Декодируем полученный двоичный код
def decoder(image_data):
    np_arr = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img



In [ ]:
# Делаем запрос и декодируем
img = decoder(requests_map(ll=ll, spn=spn, api=api_static).content)


In [ ]:
# Выводим изображение
cv2.imshow('Image_map', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Запрашиваем данные об объектах на карте

In [ ]:
# Функция для создания запроса данных с карты
def requests_data(text,types,ll,spn,api):
    response = requests.get(f'https://suggest-maps.yandex.ru/v1/suggest?text={text}%20&types={types}&ll={ll}&spn={spn}&apikey={api}&attrs=uri&print_address=1&ull={ll}&results=10')
    if response.status_code == 200:
        print("success")
        return response
    else:
        print("error:", str(response.status_code))

In [ ]:
# Функция для конвертации едениц измерения
def distance_convert(distance_str):
    if distance_str.find('км') != -1:
        distance = float(distance_str[:-3]) * 1000
    else:
        distance = float(distance_str[:-2])
    return distance

In [ ]:
df = pd.read_csv('select-database.csv')

In [ ]:
def num_(data_obj_maps):
    num = 0
    if str(data_obj_maps).find('results') != -1:
        for i in range (len(data_obj_maps['results'])):
            if distance_convert(data_obj_maps['results'][i]['distance']['text']) <= distance:
                num += 1
        else:
            return num
    else:
        num = 0
        return num
        print('None')

In [66]:

for i in range(111):
    # 2,10
    collum = ['distanceFromTheCenter', 'metro', 'cafe', 'park', 'beaches', 'hospital', 'police', 'shop', 'beautySalone', 'mall', 'museum', 'pharmacy', 'sportCenter', 'postOffice', 'hotel', 'college', 'school', 'kindergarten']
    data_arr = []
    ll = df['coordinate'][i]
    address = df['address'][i]
    
    print(address)
    response_center = json.loads(requests.get(f'https://suggest-maps.yandex.ru/v1/suggest?apikey={api_Geosajest}&text={address}&ll={CenterCoordinate}&types=house&results=1').content)
    distanceFromTheCenter = (distance_convert(response_center['results'][0]['distance']['text']))

    data_arr.append(distanceFromTheCenter)

    for keys in infoMap:
        text = infoMap[keys][0]
        types = infoMap[keys][1]
        

        data_obj_maps = json.loads(requests_data(text=text, types=types, ll=ll, spn=spn, api=api_Geosajest).content)

        # Перебираем найденные объекты и выводим их если они расположены не дальше задонного значения
        number = num_(data_obj_maps=data_obj_maps)

        data_arr.append(number)

    print('🔃', str(i+1))
    print(data_arr)

    for n in range(len(collum)):
        df_.loc[i, collum[n]] = data_arr[n]
    

    
    
        


Амурская ул.,  1Ак1
success
станция Локомотив
метро Черкизовская
success
Самая вкусная шаурма
Pandora lounge&shop
Brancho
Вкусно — и точка
Луснар
Шашлык ФУД
success
Черкизовский детский парк
Сиреневый сад
success
success
Городская поликлиника № 191, филиал № 4
success
success
success
success
Город Хобби
Мандарин
Новочеркизовский
success
Запасной командный пункт Верховного Главнокомандующего Красной Армии И.В. Сталина периода Великой Отечественной войны 1941-1945
success
Горздрав
Будь Здоров!
Ригла
Ист Фарм
Столички
success
Локо-Спорт
NeZebra
BMX-Велодром Амурский
Один
success
Отделение почтовой связи № 105122
Отделение почтовой связи № 107553
success
Левел
Отель на Щелковской
Акварель
Восточная
success
ГБПОУ Технологический колледж № 21, факультет Мегадизайн
success
Школа № 319, здание № 1
Школа № 2033, Дмитриевский корпус
Школа № 2033, Екатерининский корпус
success
Детский центр Librimi
Космо Кидс
Горница-Узорница
Школа № 319, здание № 6
Сиреневая академия
Школа № 319, здание № 7
🔃 1


C:\Users\drusl\AppData\Local\Temp\ipykernel_2332\580209301.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_.loc[i, collum[n]] = data_arr[n]


success
метро Шаболовская
success
Где Поесть
Urban Kitchen
Все ли поели?
Между Делом
The Hummus
Индийская точка
success
Прятки в темноте
success
success
Морозовская детская городская клиническая больница
Городская клиническая больница № 1 имени Н.И. Пирогова
success
Участковый пункт полиции
Участковый пункт полиции
Управление уголовного розыска ГУ МВД России по Московской области
Участковый пункт полиции № 4
Управление экономической безопасности и противодействия коррупции ГУ МВД России по г. Москве
2-й отдел полиции УВД на Московском метрополитене Главного управления МВД России по городу Москве
success
success
Салон красоты Pacho
Город красоты
Sахар
Амстердам - академия стиля Keune
success
Ереван Плаза
Тульский
success
success
Анима
Горздрав
ЛКМ-Фарма
Столички
Столички
Эконом
success
Спортивная школа олимпийского резерва № 41
The Body Cult
Bodyhacker
Идеал Фитнес
Brooklyn Fitboxing
ЦСиО Самбо-70 Москомспорта, отделение Шаболовка
success
Отделение почтовой связи № 115162
Отделение почт

KeyError: 'results'

In [79]:
(df[:52]).to_csv('full-db.csv', index=False)

In [74]:
df_ = (df[:40])

In [ ]:
print(len(data_obj_maps['results']))

In [ ]:
for i in range (len(data_obj_maps['results'])):           
    print(data_obj_maps['results'][i]['title']['text'], distance_convert(data_obj_maps['results'][i]['distance']['text']))